# Importar librerías

In [1]:
import os
import requests
import json
import pandas as pd
from pandas import json_normalize

In [2]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [4]:
import pickle

# Crear directorio

En este directorio se guardará el dataset obtenido.

In [ ]:
directory = "dataset"
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(parent_dir)

path = os.path.join(parent_dir, directory)

try:
    os.mkdir(path)
    print(f"Se ha creado el directorio {directory} en: {parent_dir}")
except OSError as error:
    print(f"El directorio {directory} en: {parent_dir} ya existe.")

# Obtención del dataset

Es equivalente a lo siguiente: `$.result.results[0].resources[1].url`

Donde `[1]` es para el csv y `[2]` para el json. Para trabajar con pandas prefiero el csv.

In [ ]:
header = {"User-Agent": "Application"}
dataset = "raw_dataset.csv"
csv_path = os.path.join(path, dataset)
year = 2016

while True:
    url = f"https://catalogodatos.cnmc.es/api/3/action/package_search?q=Precios%20diarios%20provinciales%20-%20{year}%20-"
    r = requests.get(url, headers=header)

    if r.status_code == 200:
        data_catalog = r.json()
    
        if data_catalog["result"]["results"]:
            id_resource = data_catalog["result"]["results"][0]["resources"][1]["url"]
            downloaded_file = requests.get(id_resource)
    
            with open(csv_path, "ab") as f:
                f.write(downloaded_file.content)
    
            print(f"Descargados los datos para el año {year}.")
            year += 1
        else:
            print("No hay más datos que descargar.")
            break
    else:
        print(f"No ha podido procesarse la solicitud. Código: {r.status_code}")
        break

# Dataframe

## Lectura y visualizado

In [ ]:
df_fuel = pd.read_csv(csv_path, delimiter=';')
df_fuel.head()

In [ ]:
rows, cols = df_fuel.shape
print(f"Filas: {rows} | Columnas: {cols}")

## Comprobación de nulos y tipo de datos

In [ ]:
df_fuel.isna().sum()

¡No hay nulos!

In [ ]:
df_fuel.dtypes

## Provincias

Queremos saber qué provincias hay en el dataset:

In [ ]:
df_fuel["Provincia"].unique()

Aquí se usa LabelEncoder para pasar los valores categóricos de Provincia a numéricos, necesario para el entrenamiento del modelo:

In [32]:
df_fuel["num_provincia"] = le.fit_transform(df_fuel["Provincia"])

In [ ]:
df_fuel.head()

In [ ]:
df_fuel["num_provincia"].unique()

In [ ]:
result = df_fuel.loc[df_fuel["num_provincia"] == 38]
result.head()

Puede parecer extraño pero esos valores se deben a que cuando se juntaron todos los archivos se añadió también la cabecera. Por lo tanto, las filas que sean de valor 38 las eliminamos.

In [36]:
df_fuel.drop(df_fuel[df_fuel["num_provincia"] == 38].index, inplace = True)

In [ ]:
df_fuel["num_provincia"].unique()

Se ha eliminado la etiqueta que sobra.

In [ ]:
print(le.classes_)

## Producto

De igual manera, hay que hacer lo mismo con Producto:  
En este caso uso un mapeo.

In [39]:
df_fuel["num_producto"] = df_fuel["Producto"].map({"Gasolina 95 E5":0, "Gasolina 98 E5":1, "Gasóleo A habitual":2, 
                                               "Gasóleo Premium":3})

In [ ]:
df_fuel.head()

In [ ]:
df_fuel["num_producto"].unique()

## Fechas

Se divide la fecha en tres columnas: año, mes, día.

In [42]:
df_fuel["Fecha Precio"] = pd.to_datetime(df_fuel["Fecha Precio"])

In [43]:
df_fuel["anno"] = df_fuel["Fecha Precio"].dt.year
df_fuel["mes"] = df_fuel["Fecha Precio"].dt.month
df_fuel["dia"] = df_fuel["Fecha Precio"].dt.day

In [ ]:
df_fuel.head()

## Renombrado de columnas

Nos quedamos solo con las columnas que nos interesan y renombramos algunas de ellas:

In [45]:
df_fuel = df_fuel[["anno","mes","dia","num_provincia","num_producto","Promedio de Pai Diario CUBO €/litro","Promedio de Pvp Diario CUBO €/litro"]]
df_fuel.rename(columns={"Promedio de Pai Diario CUBO €/litro": "promedio_de_pai_diario_cubo", "Promedio de Pvp Diario CUBO €/litro": "promedio_de_pvp_diario_cubo"}, inplace = True)

In [ ]:
df_fuel.head()

## Pasar valores con comas a puntos

In [47]:
df_fuel["promedio_de_pai_diario_cubo"] = df_fuel["promedio_de_pai_diario_cubo"].str.replace(',', '.').astype(float)
df_fuel["promedio_de_pvp_diario_cubo"] = df_fuel["promedio_de_pvp_diario_cubo"].str.replace(',', '.').astype(float)

In [ ]:
df_fuel.head()

In [ ]:
df_fuel.dtypes

## Guardado del dataset transformado

In [50]:
dataset = "dataset.csv"
clean_csv_path = os.path.join(path, dataset)

df_fuel.to_csv(clean_csv_path, index=False)

# Entrenamiento

En caso de ser necesario (para no tener que arrancar todo lo anterior si cerramos el archivo) cargamos de nuevo el dataset:

In [51]:
df_fuel = pd.read_csv(clean_csv_path)

In [ ]:
df_fuel.head()

## Modelo de predicción sin impuestos (PAI):

In [53]:
def train_pai():
    X = df_fuel.drop(["promedio_de_pai_diario_cubo","promedio_de_pvp_diario_cubo"], axis=1)
    y = df_fuel["promedio_de_pai_diario_cubo"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    pai = DecisionTreeRegressor()
    pai_model = pai.fit(X_train, y_train)
    model_score = pai.score(X_test, y_test)

    print(f"La puntuación del modelo es de: {model_score} %")

    return pai_model

In [ ]:
pai_model = train_pai()

## Modelo de predicción con impuestos (PVP):

In [55]:
def train_pvp():
    X = df_fuel.drop(["promedio_de_pai_diario_cubo","promedio_de_pvp_diario_cubo"], axis=1)
    y = df_fuel["promedio_de_pvp_diario_cubo"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    pvp = DecisionTreeRegressor()
    pvp_model = pvp.fit(X_train, y_train)
    model_score = pvp.score(X_test, y_test)

    print(f"La puntuación del modelo es de: {model_score} %")

    return pvp_model

In [ ]:
pvp_model = train_pvp()

## Guardado de los modelos

In [ ]:
directory = "model"
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

path = os.path.join(parent_dir, directory)

try:
    os.mkdir(path)
    print(f"Se ha creado el directorio {directory} en: {parent_dir}")
except OSError as error:
    print(f"El directorio {directory} en: {parent_dir} ya existe.")

In [59]:
def save_model(model, filename):
    directory = "model"
    path = os.path.join(parent_dir, directory)
    
    save_file = os.path.join(path, filename)

    with open(save_file, "wb") as f:
        pickle.dump(model, f)

In [60]:
save_model(pai_model, "pai_model.pkl")

In [61]:
save_model(pvp_model, "pvp_model.pkl")

Ejemplo de carga de modelo:

In [ ]:
# with open('model.pkl', 'rb') as f:
#     clf2 = pickle.load(f)

# clf2.predict(X[0:1])